In [33]:
#!pip install -qU langchain openai

We'll start by initializing the `ChatOpenAI` object. For this we'll need an [OpenAI API key](https://platform.openai.com/account/api-keys). Note that there is naturally a small cost to running this notebook due to the paid nature of OpenAI's API access.

In [34]:
import json
import openai
from langchain.chat_models import AzureChatOpenAI


with open(r'..\config.json') as config_file:
    config_details = json.load(config_file)

openai.api_base = config_details['OPENAI_API_BASE']
openai.api_key = config_details['OPENAI_API_KEY']  
openai.api_type = "azure"
openai.api_version = "2022-12-01" 


# initialize the models
chat = AzureChatOpenAI(openai_api_version="2023-05-15" , deployment_name = "gpt-35-turbo", model_name="gpt-35-turbo", openai_api_key=openai.api_key, openai_api_base=openai.api_base, temperature=0)

Initialize the `ChatOpenAI` object. We'll set `temperature=0` to minimize randomness and make outputs repeatable.

Chats with the Chat-GPT model `gpt-3.5-turbo` are typically structured like so:

```
System: You are a helpful assistant.

User: Hi AI, how are you today?

Assistant: I'm great thank you. How can I help you?

User: I'd like to understand string theory.

Assistant: 
```

The final `"Assistant:"` without a response is what would prompt the model to continue the comversation. In the official OpenAI `ChatCompletion` endpoint these would be passed to the model in a format like:

```json
[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hi AI, how are you today?"},
    {"role": "assistant", "content": "I'm great thank you. How can I help you?"}
    {"role": "user", "content": "I'd like to understand string theory."}
]
```

In LangChain there is a slightly different format. We use three *message* objects like so:

In [35]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand string theory.")
]

The format is very similar, we're just swapper the role of `"user"` for `HumanMessage`, and the role of `"assistant"` for `AIMessage`.

We generate the next response from the AI by passing these messages to the `ChatOpenAI` object.

In [36]:
res = chat(messages)
res

AIMessage(content='Sure, I can help you with that. String theory is a theoretical framework in physics that attempts to reconcile quantum mechanics and general relativity. It proposes that the fundamental building blocks of the universe are not point-like particles, but rather tiny, one-dimensional "strings" that vibrate at different frequencies. These strings are incredibly small, with a length scale of around 10^-35 meters.\n\nThe vibrations of these strings determine the properties of the particles that they form, such as their mass and charge. In this way, string theory provides a way to unify all of the fundamental forces of nature, including gravity, electromagnetism, and the strong and weak nuclear forces.\n\nOne of the key features of string theory is that it requires the existence of extra dimensions beyond the three spatial dimensions that we are familiar with. In fact, string theory requires a total of 10 or 11 dimensions, depending on the specific version of the theory.\n\n

In response we get another AI message object. We can print it more clearly like so:

In [37]:
print(res.content)

Sure, I can help you with that. String theory is a theoretical framework in physics that attempts to reconcile quantum mechanics and general relativity. It proposes that the fundamental building blocks of the universe are not point-like particles, but rather tiny, one-dimensional "strings" that vibrate at different frequencies. These strings are incredibly small, with a length scale of around 10^-35 meters.

The vibrations of these strings determine the properties of the particles that they form, such as their mass and charge. In this way, string theory provides a way to unify all of the fundamental forces of nature, including gravity, electromagnetism, and the strong and weak nuclear forces.

One of the key features of string theory is that it requires the existence of extra dimensions beyond the three spatial dimensions that we are familiar with. In fact, string theory requires a total of 10 or 11 dimensions, depending on the specific version of the theory.

While string theory is st

Because `res` is just another `AIMessage` object, we can append it to `messages`, add another `HumanMessage`, and generate the next response in the conversation.

In [38]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="Why do physicists believe it can produce a 'unified theory'?"
)
# add to messages
messages.append(prompt)

# send to chat-gpt
res = chat(messages)

print(res.content)

Physicists believe that string theory has the potential to produce a unified theory because it provides a way to reconcile the two pillars of modern physics: quantum mechanics and general relativity.

Quantum mechanics describes the behavior of particles on very small scales, while general relativity describes the behavior of gravity on large scales. However, these two theories are fundamentally incompatible with each other, and attempts to combine them have been unsuccessful so far.

String theory proposes that the fundamental building blocks of the universe are not point-like particles, but rather tiny, one-dimensional "strings" that vibrate at different frequencies. These strings are incredibly small, with a length scale of around 10^-35 meters.

The vibrations of these strings determine the properties of the particles that they form, such as their mass and charge. In this way, string theory provides a way to unify all of the fundamental forces of nature, including gravity, electrom